In [4]:
%matplotlib inline
import mxnet as mx
import numpy as np
import matplotlib.pyplot as plt



## vgg 微调

In [ ]:
sym, arg_params, aux_params = mx.model.load_checkpoint(prefix='model/vgg16', epoch=0)

## Resnet101 微调

In [14]:
sym, arg_params, aux_params = mx.model.load_checkpoint(prefix='model/resnet-101', epoch=0)

In [6]:
import mxnet as mx

mnist = mx.test_utils.get_mnist()

In [7]:
batch_size = 128
train_iter = mx.io.NDArrayIter(data=mnist['train_data'], 
                               label=mnist['train_label'], 
                               batch_size=batch_size, 
                               shuffle=True)

eval_iter = mx.io.NDArrayIter(data=mnist['test_data'], 
                              label=mnist['test_label'], 
                              batch_size=batch_size, 
                              shuffle=False)

In [8]:
sym, arg_params, aux_params = mx.model.load_checkpoint('model/resnet-152', epoch=0)

In [10]:
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:]

['bn1_moving_var',
 'bn1_output',
 'relu1_output',
 'pool1_output',
 'flatten0_output',
 'fc1_weight',
 'fc1_bias',
 'fc1_output',
 'softmax_label',
 'softmax_output']

In [14]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

net = all_layers['flatten0_output']
net = mx.sym.FullyConnected(data=net, num_hidden=10, name='fc')
net = mx.sym.SoftmaxOutput(data=net, name='softmax')

resnet_finetune = mx.mod.Module(symbol=net, context=mx.gpu())

resnet_finetune.fit(train_data=train_iter, 
                    eval_data=eval_iter, 
                    optimizer='sgd', 
                    optimizer_params={'learning_rate':0.1}, 
                    num_epoch=10, 
                    batch_end_callback=mx.callback.Speedometer(batch_size, 128))

INFO:root:Epoch[0] Batch [128]	Speed: 199.79 samples/sec	accuracy=0.558321
INFO:root:Epoch[0] Batch [256]	Speed: 201.02 samples/sec	accuracy=0.937805


KeyboardInterrupt: 

In [15]:
test_iter = mx.io.NDArrayIter(data=mnist['test_data'], 
                              label=mnist['test_label'], 
                              batch_size=batch_size, 
                              shuffle=False)

acc = mx.metric.Accuracy()
resnet_finetune.score(eval_data=test_iter, eval_metric=acc)
print(acc)

[('accuracy', 0.96855221518987344)]